In [6]:
import numpy as np
import pandas as pd
import wfdb
import os
import scipy.signal as signal
import biosppy.signals.tools as st

from scipy.signal import butter, filtfilt

from scipy.signal import butter, filtfilt, iirnotch, savgol_filter, medfilt

In [7]:
#Setting up data directories and file names
data_dir = '../base_data/'  # PhysioNet data directory
record_name = 'x35'  # Record Name to Read

record = wfdb.rdrecord(data_dir + record_name)
ecg_signal = record.p_signal.flatten()

In [8]:
# Input the sampling frequency
# In PhysioNet data, sampling frequency is 100Hz
fs = 100

In [9]:
# Set the filters you want
# 1. Apply a fourth Butterworth low-pass filter to eliminate high-frequency interference above 45 Hz
def butterworth_lowpass_filter(input_signal, fs, cutoff_freq):
    nyquist_freq = 0.5 * fs
    normal_cutoff = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normal_cutoff, btype='low', analog=False)
    filtered_signal = signal.filtfilt(b, a, input_signal)
    return filtered_signal

# 2. Filter the baseline drift or muscle noise signal through a fourth bandpass filter with a frequency band of 0.5 Hz to 10 Hz
def bandpass_filter(input_signal, fs, lowcut, highcut):
    nyquist_freq = 0.5 * fs
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = signal.butter(4, [low, high], btype='band', analog=False)
    filtered_signal = signal.filtfilt(b, a, input_signal)
    return filtered_signal

# 3. Use a Median filter to remove isolated noise
def median_filter(input_signal, kernel_size):
    filtered_signal = signal.medfilt(input_signal, kernel_size)
    return filtered_signal

In [10]:
# Filter 1
ecg_signal_filtered = butterworth_lowpass_filter(ecg_signal, fs=100, cutoff_freq=45)

# Filter 2
ecg_signal_filtered = bandpass_filter(ecg_signal_filtered, fs=100, lowcut=0.5, highcut=10)

# Filter 3
ecg_signal_filtered = median_filter(ecg_signal_filtered, kernel_size=3)

In [11]:
# Converting filtered signals to a two-dimensional array
filtered_signal = np.reshape(ecg_signal_filtered, (-1, 1))

# Set the file name to store the data
new_record_name = 'x35'

# Save the data
wfdb.wrsamp(new_record_name, fs=fs, units=['mV'], fmt=['16'], sig_name=['ECG'], p_signal=filtered_signal)